In [2]:
!pip install pyparsing mortardata pyarrow "pandas[performance]>=2.0"

  Obtaining dependency information for pyparsing from https://files.pythonhosted.org/packages/39/92/8486ede85fcc088f1b3dba4ce92dd29d126fd96b0008ea213167940a2475/pyparsing-3.1.1-py3-none-any.whl.metadata
  Using cached pyparsing-3.1.1-py3-none-any.whl.metadata (5.1 kB)
  Obtaining dependency information for mortardata from https://files.pythonhosted.org/packages/78/68/131b2b35464eb49dc00b15810d0d82c5be4bd2457f72562eb5d6d285cd01/mortardata-0.1.5-py3-none-any.whl.metadata
  Using cached mortardata-0.1.5-py3-none-any.whl.metadata (2.1 kB)
  Obtaining dependency information for pyarrow from https://files.pythonhosted.org/packages/9a/bf/e88a36fa1669a5ed3a222bb6f8dee25627c13ee2c9297fd9d9e9dc0910e9/pyarrow-13.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata
  Using cached pyarrow-13.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
  Obtaining dependency information for pandas[performance]>=2.0 from https://files.pythonhosted.org/packages/d9/26/895a49ebddb4211f2d777150f38ef9e538def

In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np
import pandas as pd
from itertools import product
import time
import os

This cell downloads a large dataset for us to practice with. The details for this cell are not super important, but if you want to run it from somewhere else other than hub.mortardata.org, let me now. This cell only needs to be run once. After you run it, be sure to load the dataset from the resulting CSV file.

In [2]:
if not os.path.exists("vav_data.csv"):
    from mortardata import Client
    c = Client()
    print(c.sites)
    vav_points = """
    PREFIX brick: <https://brickschema.org/schema/Brick#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
    SELECT ?point ?id WHERE {
        ?point rdf:type/rdfs:subClassOf* brick:Temperature_Sensor .
        ?point ref:hasExternalReference/ref:hasTimeseriesId ?id .
    }"""
    c.data_sparql_to_csv(vav_points, "vav_data.csv", start="2016-01-01", end="2016-06-01", sites=c.sites[:10])
df = pd.read_csv("vav_data.csv", dtype_backend="pyarrow")
df.describe()

,77.04264831542969
count,4.876457e+07
mean,7.223155e+01
std,5.147247e+01
min,-1.240000e+00
25%,6.930000e+01
50%,7.246367e+01
75%,7.460055e+01
max,9.999000e+03


In [4]:
# create a table from the dataframe so we can give it to parquet
table = pa.Table.from_pandas(df)

In [5]:
class ParquetFileConfigurationTester:
    # define the options we want to try for parquet encoding
    parquet_version = "2.6" # use the newer version
    # options defined here: https://arrow.apache.org/docs/python/generated/pyarrow.parquet.write_table.html#pyarrow.parquet.write_table
    parquet_options = {
        "data_page_size": [int(i*1024*1024*1024) for i in range(1,5)],
        "row_group_size": [int(i*1024*1024) for i in range(1, 5)],
        "use_dictionary": [True, False],
        # TODO: figure out how to handle the other options!
    }
    def __init__(self, table):
        self.table = table
        self.data = pd.DataFrame(columns=["filesize", "write_time", *self.parquet_options.keys()])
        
    def create_dataset_configurations(self):
        for options in product(*self.parquet_options.values()):
            config = dict(zip(self.parquet_options.keys(), options))
            yield config
            
    def measure_dataset_configuration(self, config):
        # write the parquet file
        t0 = time.time()
        pq.write_table(table, "output.parquet", version=self.parquet_version, **config)
        # measure time to create the file
        write_time = time.time() - t0
        # measure the filesize
        filesize = os.path.getsize("output.parquet")

        row = pd.Series({"write_time": write_time,
                          "filesize": filesize,
                          **config})
        self.data = pd.concat([self.data, row.to_frame().T], ignore_index=True)

    def run_benchmarks(self):
        for config in self.create_dataset_configurations():
            self.measure_dataset_configuration(config)

In [6]:
b = ParquetFileConfigurationTester(table)
b.run_benchmarks()

In [7]:
b.data.sort_values('filesize')

,filesize,write_time,data_page_size,row_group_size,use_dictionary
18,375244350,14.954905,3221225472,2097152,True
2,375244350,15.83801,1073741824,2097152,True
26,375244350,16.187713,4294967296,2097152,True
10,375244350,15.955444,2147483648,2097152,True
0,375958488,14.862537,1073741824,1048576,True
8,375958488,16.53395,2147483648,1048576,True
24,375958488,15.668416,4294967296,1048576,True
16,375958488,15.395171,3221225472,1048576,True
6,378582076,16.037552,1073741824,4194304,True
30,378582076,16.22991,4294967296,4194304,True
